In [1]:
import numpy as np
import optuna

In [7]:
np.random.seed(0)
shape = (100, 100, 100)
tensor = np.random.randint(2, size=shape)
print(tensor)

[[[0 1 1 ... 1 1 0]
  [1 0 0 ... 1 0 0]
  [1 0 1 ... 0 1 1]
  ...
  [1 0 1 ... 1 1 1]
  [1 1 1 ... 1 0 0]
  [1 1 1 ... 0 1 0]]

 [[1 1 1 ... 1 0 1]
  [0 0 0 ... 0 0 1]
  [0 1 1 ... 0 1 1]
  ...
  [1 1 0 ... 0 0 1]
  [1 1 0 ... 1 0 0]
  [1 0 0 ... 0 0 0]]

 [[0 0 1 ... 0 0 0]
  [1 1 1 ... 1 1 1]
  [0 1 0 ... 1 0 1]
  ...
  [0 0 1 ... 1 1 0]
  [1 0 1 ... 1 0 0]
  [0 0 0 ... 0 0 1]]

 ...

 [[0 1 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 1 0]
  ...
  [1 0 1 ... 1 0 1]
  [1 0 1 ... 0 0 1]
  [1 1 0 ... 1 1 0]]

 [[0 0 0 ... 1 0 0]
  [0 1 0 ... 0 1 0]
  [1 0 0 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 0 1 ... 0 1 1]
  [1 0 0 ... 0 0 0]]

 [[1 1 1 ... 1 0 0]
  [1 1 0 ... 1 1 0]
  [0 0 0 ... 1 0 1]
  ...
  [1 0 1 ... 0 1 1]
  [1 1 0 ... 0 1 0]
  [0 1 1 ... 0 0 1]]]


In [11]:
import numpy as np

# データ生成
np.random.seed(0)
shape = (7,) * 4 
tensor = np.random.randint(2, size=shape)

# 0 のエントリのインデックスを取得
indices = np.array(np.where(tensor == 0)).T

# 選択する個数を指定
num_samples = 10

# ランダムに選択
if num_samples > len(indices):
    raise ValueError("選択する個数が 0 のエントリ数を超えています")

random_indices = indices[np.random.choice(len(indices), size=num_samples, replace=False)]

print(random_indices)


[[4 5 5 2]
 [4 4 3 4]
 [6 5 3 4]
 [2 6 1 5]
 [3 5 1 3]
 [1 0 5 3]
 [3 6 5 3]
 [0 3 1 0]
 [0 3 0 5]
 [1 2 3 2]]


In [10]:
random_indices[0]

array([81, 49, 61])

In [ ]:
class ConstraintWarcraft:
    def __init__(self, map_shape: tuple[int, int]) -> None:
        self.map_shape = map_shape
        self.directions_dict = {
            "oo": np.array([0, 0]),
            "ab": np.array([1, 1]),
            "ac": np.array([0, 2]),
            "ad": np.array([1, 1]),
            "bc": np.array([1, 1]),
            "bd": np.array([2, 0]),
            "cd": np.array([1, 1]),
        }
        self.directions_list = list(self.directions_dict.keys())

        self.tensor_constraint = self._build()

    def _build(self) -> np.ndarray:
        directions_dict = self.directions_dict
        directions_list = self.directions_list

        # Map parameters
        map_length = self.map_shape[0] * self.map_shape[1]
        ideal_gain = (self.map_shape[0] + self.map_shape[1] - 1) * 2

        # Initialize constraints as NumPy arrays
        tensor_constraint_1 = np.zeros((len(directions_list),) * map_length)
        tensor_constraint_2 = np.zeros((len(directions_list),) * map_length)
        tensor_constraint_3 = np.zeros((len(directions_list),) * map_length)

        # Constraint 1: (0, 0) != "oo", "ab"
        for direction in directions_list:
            if direction not in ["oo", "ab"]:
                # tensor_constraint_1[..., directions_list.index(direction)] = 1
                tensor_constraint_1[directions_list.index(direction), ...] = 1

        # Constraint 2: (map_shape[0] - 1, map_shape[1] - 1) != "oo", "cd"
        for direction in directions_list:
            if direction not in ["oo", "cd"]:
                # tensor_constraint_2[directions_list.index(direction), ...] = 1
                tensor_constraint_2[..., directions_list.index(direction)] = 1

        # Constraint 3: len[path] == map_shape[0] * map_shape[1]
        for index, _ in np.ndenumerate(tensor_constraint_3):
            gain = np.sum([directions_dict[directions_list[idx]].sum() for idx in index])
            if gain == ideal_gain:
                tensor_constraint_3[index] = 1

        # Combine constraints with logical AND
        tensor_constraint = np.logical_and(
            tensor_constraint_1,
            np.logical_and(tensor_constraint_2, tensor_constraint_3)
        )

        self.tensor_constraint = tensor_constraint
    
    def sample_violation_indices(self, num_samples: int) -> np.ndarray:
        indices = np.array(np.where(self.tensor_constraint == 0)).T

        if num_samples > len(indices):
            raise ValueError("num_samples is too large")

        random_indices = indices[np.random.choice(len(indices), size=num_samples, replace=False)]

        return random_indices
    
    def sample_violation_path(self, num_samples: int = 200) -> list[list[str]]:
        random_indices = self.sample_violation_indices(num_samples)
        random_paths = [list(self.directions_list[idx] for idx in index) for index in random_indices]

        return random_paths
    
    
map_shape = (2, 3)
constraint_builder = ConstraintWarcraft(map_shape)
tensor_constraint = constraint_builder.tensor_constraint
violation_paths = constraint_builder.sample_violation_path(10)
print(violation_paths)

In [27]:
map_shape = (2, 3)
constraint_builder = ConstraintWarcraft(map_shape)
tensor_constraint = constraint_builder.tensor_constraint
violation_paths = constraint_builder.sample_violation_path(10)
print(violation_paths)

ValueError: Calling nonzero on 0d arrays is not allowed. Use np.atleast_1d(scalar).nonzero() instead. If the context of this error is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.

In [ ]:
import numpy as np


class ConstraintWarcraft:
    def __init__(self, map_shape: tuple[int, int]) -> None:
        self.map_shape = map_shape
        self.directions_dict = {
            "oo": np.array([0, 0]),
            "ab": np.array([1, 1]),
            "ac": np.array([0, 2]),
            "ad": np.array([1, 1]),
            "bc": np.array([1, 1]),
            "bd": np.array([2, 0]),
            "cd": np.array([1, 1]),
        }
        self.directions_list = list(self.directions_dict.keys())

        self.tensor_constraint = self._build()

    def _build(self) -> np.ndarray:
        directions_dict = self.directions_dict
        directions_list = self.directions_list

        # Map parameters
        map_length = self.map_shape[0] * self.map_shape[1]
        ideal_gain = (self.map_shape[0] + self.map_shape[1] - 1) * 2

        # Initialize constraints as NumPy arrays
        tensor_constraint_1 = np.zeros((len(directions_list),) * map_length)
        tensor_constraint_2 = np.zeros((len(directions_list),) * map_length)
        tensor_constraint_3 = np.zeros((len(directions_list),) * map_length)

        # Constraint 1: (0, 0) != "oo", "ab"
        for direction in directions_list:
            if direction not in ["oo", "ab"]:
                tensor_constraint_1[directions_list.index(direction), ...] = 1

        # Constraint 2: (map_shape[0] - 1, map_shape[1] - 1) != "oo", "cd"
        for direction in directions_list:
            if direction not in ["oo", "cd"]:
                tensor_constraint_2[..., directions_list.index(direction)] = 1

        # Constraint 3: len[path] == map_shape[0] * map_shape[1]
        for index, _ in np.ndenumerate(tensor_constraint_3):
            gain = np.sum([directions_dict[directions_list[idx]].sum() for idx in index])
            if gain == ideal_gain:
                tensor_constraint_3[index] = 1

        # Combine constraints with logical AND
        tensor_constraint = np.logical_and(
            tensor_constraint_1,
            np.logical_and(tensor_constraint_2, tensor_constraint_3)
        )

        return tensor_constraint  

    def sample_violation_indices(self, num_samples: int) -> np.ndarray:
        indices = np.array(np.where(self.tensor_constraint == 0)).T

        if num_samples > len(indices):
            raise ValueError("num_samples is too large")

        random_indices = indices[np.random.choice(len(indices), size=num_samples, replace=False)]

        return random_indices

    def sample_violation_path(self, num_samples: int = 200) -> list[list[str]]:
        random_indices = self.sample_violation_indices(num_samples)
        random_paths = [list(self.directions_list[idx] for idx in index) for index in random_indices]

        return random_paths


# 動作確認
map_shape = (2, 3)
constraint_builder = ConstraintWarcraft(map_shape)
tensor_constraint = constraint_builder.tensor_constraint
violation_paths = constraint_builder.sample_violation_path(10)
print(violation_paths)


[['bc', 'ad', 'bc', 'oo', 'ad', 'cd'], ['bd', 'ac', 'oo', 'ac', 'cd', 'ad'], ['cd', 'oo', 'bc', 'ab', 'ad', 'ac'], ['ab', 'bc', 'oo', 'ad', 'ac', 'oo'], ['ab', 'ab', 'ad', 'ab', 'bc', 'ac'], ['oo', 'ab', 'ab', 'ad', 'bd', 'ab'], ['cd', 'ac', 'bd', 'bd', 'ac', 'ad'], ['bc', 'oo', 'ac', 'bc', 'ab', 'cd'], ['oo', 'ab', 'cd', 'ab', 'bd', 'ab'], ['ac', 'ac', 'cd', 'oo', 'cd', 'ad']]


In [25]:
import optuna
from optuna.distributions import FloatDistribution


def objective(trial):
    x = trial.suggest_float("x", 0, 10)
    return x**2


# Study の作成
study = optuna.create_study()
print("Initial study trials:", len(study.trials))  # 初期状態の試行数 (0)

# 手動で試行を作成して追加
trial = optuna.trial.create_trial(
    params={"x": 2.0},  # パラメータ
    distributions={"x": FloatDistribution(0, 10)},  # 分布
    value=4.0,  # 評価値
)
study.add_trial(trial)

print("Study trials after adding one manually:", len(study.trials))  # 試行数 (1)

# 最適化を実行
study.optimize(objective, n_trials=3)
print("Study trials after optimization:", len(study.trials))  # 試行数 (4)

# 他の Study を作成してコピー
other_study = optuna.create_study()

for trial in study.trials:
    other_study.add_trial(trial)
print("Other study trials after copying:", len(other_study.trials))  # 試行数 (4)

# コピー後に最適化
other_study.optimize(objective, n_trials=2)
print("Other study trials after additional optimization:", len(other_study.trials))  # 試行数 (6)

# 最適化結果を表示
print("\nFinal results:")
print("Original Study:")
print("Best value:", study.best_value)
print("Best params:", study.best_params)

print("\nOther Study:")
print("Best value:", other_study.best_value)
print("Best params:", other_study.best_params)


[I 2024-12-08 13:47:20,201] A new study created in memory with name: no-name-4ac8927a-72cc-4a84-acef-42030d15c73f


[I 2024-12-08 13:47:20,215] Trial 1 finished with value: 89.99921698615151 and parameters: {'x': 9.486791711962033}. Best is trial 0 with value: 4.0.
[I 2024-12-08 13:47:20,217] Trial 2 finished with value: 6.5831489371383975 and parameters: {'x': 2.565764786011843}. Best is trial 0 with value: 4.0.
[I 2024-12-08 13:47:20,219] Trial 3 finished with value: 36.07189219298212 and parameters: {'x': 6.005988028041857}. Best is trial 0 with value: 4.0.
[I 2024-12-08 13:47:20,221] A new study created in memory with name: no-name-23c7b2e1-d298-4860-93a5-be9d3e68ad16
[I 2024-12-08 13:47:20,223] Trial 4 finished with value: 9.619736123541157 and parameters: {'x': 3.1015699449699916}. Best is trial 0 with value: 4.0.
[I 2024-12-08 13:47:20,223] Trial 5 finished with value: 38.96902557257843 and parameters: {'x': 6.242517566861821}. Best is trial 0 with value: 4.0.


Initial study trials: 0
Study trials after adding one manually: 1
Study trials after optimization: 4
Other study trials after copying: 4
Other study trials after additional optimization: 6

Final results:
Original Study:
Best value: 4.0
Best params: {'x': 2.0}

Other Study:
Best value: 4.0
Best params: {'x': 2.0}
